# Google Colab Notebook for Gurobi Optimization

### Data Storage to Google Drive

In [ ]:
from google.colab import drive
import os
# mount for Google Drive
drive.mount('/content/drive', force_remount=True)
os.chdir('/content/drive/MyDrive/OPF_ML_Colab')
!ls

### Format Data to fit Gurobi

In [ ]:
# Case where number of generators per bus == 1
# Information on dimensions: B, x and P_max: N_bus x N_bus, c, P_D, P_min_G, P_max_G: N_bus x N_gen

import pandas as pd
import numpy as np

# Read Files
busses = pd.read_excel('Busses_Timeseries_Table_Set2020_ExcelEdit.xlsx')
edges = pd.read_excel('Edges_Timeseries_Table_Set2020_ExcelEdit.xlsx')

In [ ]:
# Copyright 2022, Nadine Angermeier (nadine.angermeier@tum.de), TUM

#N_bus
N_bus = busses['bus_id'].unique().size

# N_gen
# Depends on assumption, to receive same dimensions set to 1 or 2 (if split between price-sensitive and price-inelastic)
N_gen = 1

# c, P_min_G and P_max_G
# Function to generatte on set of c, P_min_G and P_max G at one timeinterval
def generate_Time_Data (time, N_bus):
    c = np.zeros((N_bus, 1))
    h = np.zeros((N_bus, 1))
    P_min_G = np.zeros((N_bus, 1))
    P_max_G = np.zeros((N_bus, 1))
    
    bussesAct = busses.loc[busses['datetime_beginning_utc'] == time]
    bussesAct.reset_index(inplace = True, drop = True)
    N_bus = len(bussesAct.index)
    
    c = bussesAct['incremental_price']
    h = bussesAct['inter_start_cost']
    P_min_G = bussesAct['min_ecomin']
    P_max_G = bussesAct['max_ecomax']
    P_D = bussesAct['mw_norm']
    
    B = np.zeros((N_bus, N_bus))
    P_max = np.zeros((N_bus, N_bus))
    
    edgesAct = edges.loc[edges['datetime_beginning_utc'] == time]
    edgesAct.reset_index(inplace = True, drop = True)
    N_edge = len(edgesAct.index)
    
    for i in range(N_edge):
        fromEdge = edgesAct.loc[i, 'From Number']
        toEdge = edgesAct.loc[i, 'To Number']
        B[fromEdge, toEdge] = 1 / edgesAct.loc[i, 'X'] #Test
        B[toEdge, fromEdge] = 1 / edgesAct.loc[i, 'X'] #Test
        P_max[fromEdge, toEdge] = edgesAct.loc[i, 'Lim MVA A']
        P_max[toEdge, fromEdge] = edgesAct.loc[i, 'Lim MVA A']
        
    return c, h, P_min_G, P_max_G, P_D, B, P_max




### Optimize with gurobipy

In [ ]:
!pip install gurobipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 43.2 MB/s eta 0:00:00


In [ ]:
# Guide on https://www.gurobi.com/documentation/10.0/quickstart_windows/cs_python.html#section:Python

# Verison 1.1.1
# Assumption: 1 generator and 1 demander per bus
# If variable number of generators per bus (even demanders per bus), need to give branch data and add each constraint

#Gurobi python package (pip install)
import gurobipy as gp
from gurobipy import *

# This code formulates and solves the following DCOPF model:
#  minimize
#        Sum[i=1 to N_bus] (ci * P_Gi + hi * ui)
#  subject to
#        P_min_Gi * ui <= P_Gi, i=1,...N_bus  c1
#        P_max_Gi * ui >= P_Gi, i=1,...N_bus  c2
#        P_Gi - P_Di = Sum[j=1 to N_bus] (Bij * (thetai - thetaj), i=1,...N_bus   c3
#        Bij * (thetai - thetaj) <= P_maxij, i,j = 1,...N_bus   c4
# Information on dimensions: B, P_max: N_bus x N_bus; P_G, c, h, P_min_G, P_max_G, P_D, theta: N_bus x 1

# Constants/ data is given, np.arrays
def run_dcopf(N_bus, c, h, P_min_G, P_max_G, B, P_D, P_max):
    try:

        # Create a new model
        m = gp.Model("dcopf")

        # Create Matrix variables
        P_G = m.addMVar(shape=N_bus, vtype=GRB.CONTINUOUS, name="P_G")
        u = m.addMVar(shape=N_bus, vtype=GRB.BINARY, name="u")
        theta = m.addMVar(shape=N_bus, vtype=GRB.CONTINUOUS, name="theta")
        
        # Set objective
        m.setObjective(gp.quicksum(c[i] * P_G[i] + h[i] * u[i] for i in range(N_bus)), GRB.MINIMIZE)

        # Add constraints
        m.addConstrs((P_min_G[i] * u[i] <= P_G[i] for i in range(N_bus)), "c1")
        m.addConstrs((P_max_G[i] * u[i] >= P_G[i] for i in range(N_bus)), "c2")
        m.addConstrs((P_G[i] - P_D[i] == gp.quicksum(B[i, j] * (theta[i] - theta[j]) for j in range(N_bus)) for i in range(N_bus)), "c3")
        m.addConstrs((B[i, j] * (theta[i] - theta[j]) <= P_max[i, j] for i in range(N_bus) for j in range(N_bus)), "c4")
        
        # Optimize model
        m.optimize()
        
        # Relaxation, not used
        #if m.status == GRB.INFEASIBLE:
            #m.feasRelaxS(1,False,False,True)
            #m.optimize()
            
        all_vars = m.getVars()    
        values = m.getAttr("X", all_vars)
        names = m.getAttr("VarName", all_vars)
        #for name, val in zip(names, values):
            #print(f"{name} = {val}")
            
        #Return result
        return m #m.getVars(), m.ObjVal
        
    except gp.GurobiError as e:
        print('Error code ' + str(e.errno) + ': ' + str(e))

    except AttributeError:
        print('Encountered an attribute error')

# Function to print results
def print_result(variables, objVal):
    for v in variables:
        print('%s %g' % (v.VarName, v.X))

    print('Obj: %g' % objVal)
        



In [ ]:
def save_result(result, timeInterval):
    variables = result.getVars()
    values = result.getAttr("X", variables)
    names = result.getAttr("VarName", variables)
    
    for name, val in zip(names, values):
        #print(name)
        index = int(re.findall('[0-9]+', name)[0])
        if (name.startswith("P_G")):
            busses.at[timeInterval * N_bus + index, 'solGenerate'] = val
        if (name.startswith("u")):
            busses.at[timeInterval * N_bus + index, 'solOn'] = val
        if (name.startswith("theta")):
            busses.at[timeInterval * N_bus + index, 'solTheta'] = val
            
    #Save inbetween as kernel dies sometimes  
    busses.to_csv("Busses_Timeseries_Table_Set2020_Solution_Complete.csv")

# Empty/ Infeasible rows need to be deleted, Demanders and generators need to be stored in csv

In [ ]:
# Go through data, optimize and print/ store                                           
timestamps = edges['datetime_beginning_utc'].unique()

for timeInterval in range(len(timestamps)): 
    time = pd.Timestamp(timestamps[timeInterval])
    c, h, P_min_G, P_max_G, P_D, B, P_max = generate_Time_Data (time, N_bus)
    result = run_dcopf(N_bus, c, h, P_min_G, P_max_G, B, P_D, P_max)
    if result == None:
        print('Infeasible')
    else: 
        # Feasible        
        save_result(result, timeInterval)
    
    